This is an implementation of https://drive.google.com/file/d/1DtYllaV4Yk8ljgYcLBmdXNplEDTG6HT6/view

In [1]:
# Warmup
import numpy as np
import torch
import pdb

In [2]:
device = torch.device("cuda:0")

In [3]:
def sample_data():
    
    count = 10000
    rand = np.random.RandomState(0)
    a = 0.3 + 0.1 * rand.randn(count)
    b = 0.8 + 0.05 * rand.randn(count)
    mask = rand.rand(count) < 0.5
    samples = np.clip(a * mask + b * (1 - mask), 0.0, 1.0)
    
    return np.digitize(samples, np.linspace(0.0, 1.0, 100))

In [4]:
dataset = sample_data()

In [5]:
dataset_train = dataset[:np.int32(len(dataset) * 80 / 100)]
dataset_valid = dataset[np.int32(len(dataset) * 80 / 100):]

In [6]:
len(dataset_train) + len(dataset_valid)

10000

In [7]:
# define the model (softmax)
# define the loss (maximum likelihood)
input = torch.randn(2, 3); print(input)

tensor([[-0.9149,  0.2920, -0.4746],
        [-0.5851, -0.7474,  0.5795]])


for the Negative log likelihood check the following: 
http://bjlkeng.github.io/posts/autoregressive-autoencoders/

In [9]:
# learning rate
learning_rate = 5.0e-1

# total number of samples
n_dataset = len(dataset_train)

# number of samples
n_samples = 100

# set weights to 0 - with torch grad! check this
thetas = torch.zeros(n_samples, dtype=torch.float32, requires_grad=True)

# negative log likelihood
# loss = torch.nn.NLLLoss()

# loop: grab 100 samples from training data ( == target) untill you consume it
for i in range(10000):  
    
    random_datapoint = np.random.choice(dataset_train, size=1024)

    # extract index of sample
    inx_sample = torch.from_numpy(random_datapoint)
#     pdb.set_trace()

    # calculate distribution of target using the softmax formula (== convert to a proba distribution)
#     Pdata = sample / torch.sum(sample)
    
    # calculate distribution of  model using the softmax formula + initiate theta (weights) to 0
    Pmodel = torch.exp(thetas) / torch.sum(torch.exp(thetas))
    
    # calculate loss (KL divergence = log likelihood loss...)
    # can be done: loss = torch.sum((Pmodel - Pdata) * (Pmodel - Pdata)) / n_samples
    # must give the same result
    # check cross entropy loss as well!
#     pdb.set_trace()
#     loss = torch.nn.MSELoss(Pmodel, Pdata)  
#     loss = torch.sum((Pmodel - Pdata) * (Pmodel - Pdata)) / n_samples
    
    loss = torch.sum(-torch.log2(Pmodel[inx_sample])) / len(random_datapoint)
    
    # backprop
    loss.backward()
    
    pdb.set_trace()
    
    # print the loss
    print('loss = ', loss.item())
    
    # backward
    # update sgd params
    with torch.no_grad():
    
        # SGD
#         pdb.set_trace()
        thetas -= learning_rate * thetas.grad
        
        # manually zero gradients after updating the weights
        thetas.grad.zero_()

> <ipython-input-9-867944c6dda6>(47)<module>()
-> print('loss = ', loss.item())
(Pdb) n
loss =  6.643856048583984
> <ipython-input-9-867944c6dda6>(51)<module>()
-> with torch.no_grad():
(Pdb) thetas.grad
(Pdb) l
 46  	    # print the loss
 47  	    print('loss = ', loss.item())
 48  	
 49  	    # backward
 50  	    # update sgd params
 51  ->	    with torch.no_grad():
 52  	
 53  	        # SGD
 54  	#         pdb.set_trace()
 55  	        thetas -= learning_rate * thetas.grad
 56  	
(Pdb) q


BdbQuit: 

In [12]:
for i in range(5000):
    
    random_datapoint = np.random.choice(dataset_train, size=1024)
    pdb.set_trace()

> <ipython-input-12-6137cdd0f257>(1)<module>()
-> for i in range(5000):
(Pdb) dataset_train
array([48, 76, 88, ..., 80, 28, 80])
(Pdb) q


BdbQuit: 

In [21]:
a

tensor([-0.5224, -2.1991, -0.2925, -0.2785,  1.4339, -0.4543, -1.4431, -0.2508,
         0.0749,  0.0203])

In [22]:
inx = torch.from_numpy(np.array([2, 4, 6 ,7, 9, 0, 1, 3, 5, 8]))

In [23]:
a[inx]

tensor([-0.2925,  1.4339, -1.4431, -0.2508,  0.0203, -0.5224, -2.1991, -0.2785,
        -0.4543,  0.0749])